<a href="https://colab.research.google.com/github/whiteHatpro/coursera_algorithmic_toolbox/blob/master/NLP_Data_Extractor(3)_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Backend Clean code


## Imports

In [ ]:
!pip install wget | grep -v 'already satisfied' #(added section to remove clutter of dependancies alredy installed)
!pip install PyPDF2 | grep -v 'already satisfied'
!pip install textract | grep -v 'already satisfied'
!pip install PyMuPDF | grep -v 'already satisfied'

In [ ]:
import pandas as pd
import bs4
import wget
import fitz
import numpy as np
import PyPDF2
import textract
import re
from PyPDF2 import PdfReader
from nltk.corpus import stopwords
import string
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer
import nltk
import urllib.request
from urllib.request import urlopen
from urllib.request import urlparse
from ctypes.util import find_library
import csv
from google.colab import drive
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
#DATA EXTRACTION BLOCK:

#extracting the title and page number
def titel(file):
    reader = PdfReader(file)
    meta = reader.metadata
    #print(len(reader.pages))
    #print(meta.title)
    return meta,len(reader.pages)
#titel(file)

#extract the text from the pdf
def text(file):
    pdfReader = PyPDF2.PdfReader(file)
    num_pages = len(pdfReader.pages)

    count = 0
    text = ""

    while count < num_pages:
        #error pageObj = pdfReader.getPage(count)
        pageObj = pdfReader.pages[count]
        count +=1
        text += pageObj.extract_text()

    if text != "":
        text = text

    else:
        text = textract.process('words.txt', method='tesseract', language='eng')


    text = text.encode('ascii','ignore').lower()
    text_decoded = text.decode()
    return text_decoded

#Pagewise Implimentation


def extract_page(file,num=0):
    pdf_file = fitz.open(file)
    page_texts = []

    for page_number in range(len(pdf_file)):
        page = pdf_file.load_page(page_number)
        page_text = page.get_text()

        # Encode the page text as ASCII and convert to lowercase
        page_text = page_text.encode('ascii', 'ignore').lower()
        page_text_decoded = page_text.decode()
        page_texts.append(page_text_decoded)

    #page=int(input("Enter Page Number: "))
    filepage = ""

    if page_texts:
      filepage = page_texts[num]


    return filepage


In [ ]:
#DATA REFINEMENT BLOCK

def cleanText(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = text.replace("/", " ")
    text = text.replace("\n", " ")
    text = text.replace("''", "")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text
round1 = lambda x: cleanText(x)

#Remove stopwords
stopwords_en = stopwords.words('english')
additionalStopWords = ["pg","figure","u"]    #Remove the words that are present in the title
stopwords_en.extend(additionalStopWords)
def toggleStopword(newStopword):
  if newStopword in stopwords_en:
        stopwords_en.remove(newStopword)
  else:
        stopwords_en.append(newStopword)

def removeStopwords(texts):
    extract = ''
    for word in simple_preprocess(texts):
        if word not in stopwords_en:
            extract = extract + ' ' + word
    return extract

#Lemmatize keywords
def lemmatizeKeywords(keywords):
    keywords_list = keywords.split(' ')
    lemmatizer = WordNetLemmatizer()
    keywords_lemmatized = []
    for keyword in keywords_list:
        keywords_lemmatized.append(lemmatizer.lemmatize(keyword))
    return ' '.join(keywords_lemmatized)


def identifyFrequentKeywords(keywordsFlat,additionalWords=[""]): #Additional words should be a list of strings.
    keywords = keywordsFlat.split(' ')
    while '' in keywords:
        keywords.remove('')
    keywordsFlat = ' '.join(keywords)
    keywordCount = len(keywords)

    trigrams_keys = []
    trigrams_values = []
    trigrams = nltk.trigrams(keywords)
    frequency = nltk.FreqDist(trigrams)
    for key,value in frequency.items():
        if value > 1: #value->absolute count
            trigrams_keys.append(' '.join(key))
            trigrams_values.append(value/keywordCount) #->persisting relative frequency

    bigrams_keys = []
    bigrams_values = []
    bigrams = nltk.bigrams(keywordsFlat.split(' '))
    frequency = nltk.FreqDist(bigrams)
    for key,value in frequency.items():
        if value > 1:
            bigrams_keys.append(' '.join(key))
            bigrams_values.append(value/keywordCount)

    monograms_keys = []
    monograms_values = []
    frequency = nltk.FreqDist(keywordsFlat.split(' '))
    for key,value in frequency.items():
        if value > 1:
            monograms_keys.append(key)
            monograms_values.append(value/keywordCount)

    additionalWords_words=[]
    additionalWords_values=[]
    additionalWords=[word.lower() for word in additionalWords]
    frequency = nltk.FreqDist(keywordsFlat.split(' '))
    for key,value in frequency.items():
      if key in additionalWords:
        if value> 1:
          #print(key,value)
          additionalWords_words.append(key)
          additionalWords_values.append(value/keywordCount)
      # for
      #   additionalWords_values.append(["Not Present"])
        #print(additionalWords_values)


    keys_selected = trigrams_keys + bigrams_keys + monograms_keys
    keys_selected_values = trigrams_values + bigrams_values + monograms_values
    keys_selected = [x for _, x in sorted(zip(keys_selected_values, keys_selected), reverse=True)]
    keys_selected_values.sort(reverse=True)

    #select top-n keywords
    numberOfTopWords = 50
    if len(keys_selected) > numberOfTopWords:
        keys_selected        = keys_selected[:numberOfTopWords]  + additionalWords_words
        keys_selected_values = keys_selected_values[:numberOfTopWords] + additionalWords_values

    return keys_selected, keys_selected_values, keywordCount

additionalKeywords=[""]

def toggleKeyword(newKeyword):
  if newKeyword in additionalKeywords:
        additionalKeywords.remove(newKeyword)
  else:
        additionalKeywords.append(newKeyword)

def keywordExtractor(extractedPage):
      x=extractedPage
      x=cleanText(x)
      x=lemmatizeKeywords(x)                                  #Turn words into their root ( Happily to Happy, faster to fast, etc)
      x=removeStopwords(x)                                    #Only to be used if there's any specific words we want to remove
      m=identifyFrequentKeywords(x,additionalKeywords)        #Extracts the CSV style entry in format [Word,Freq,PageWordCount] of our page's frequent keywords, we can add additional keywords to check as well if we wish
      return m

In [ ]:
#WORD BASED FUNCTIONS

def word_count(word, text1):                                      #Count of a specific word in the document
    with open('/content/drive/MyDrive/test6.txt','w+') as f:
        #f.write(cleanText(text(file1)))
        f.write(text1)
    with open('/content/drive/MyDrive/test6.txt') as f:
        return ''.join(f).count(word)

def word_cooccurrence(word1,word2,file):                        #Extracts list of pages where words 1 and 2 appear
  page_list=[]
  pages=titel(file)[1]
  for i in range(pages):
    n=extract_page(file,i)
    n=n.lower()
    if word1.lower() in n:
      if word2.lower() in n:
        page_list.append(i)
  return page_list                                            #REMEMBER, this is the list of pages from index 0

def wordTotal(pageList,file):                               #Extracts total number of words on the pages we are working on
  wordTotal=0
  for i in pageList:
      n=extract_page(file,i)
      m=keywordExtractor(n)
      a,b,c=m
      wordTotal+=c
  return wordTotal



In [ ]:
#FREQUENCY CSV FUNCTIONS
def CSVFreq(file):                                           #Get the total CSV of frequent keywords and their frequencies for our file
  pages=titel(file)[1]
  CSV=[]
  for i in range(pages):
      n=extract_page(file,i)
      m=keywordExtractor(n)
      a,b,c=m
      print(i+1)
      print(m)
      for j in range(len(a)):
          CSV.append([i+1,a[j],b[j],c])                    #Outputs CSV format [Page,Word,Freq,PageWordCount]
  return CSV

def associatedCSVFrequency(pageList, file):                 #Get the CSV of frequent keywords and their frequencies for a specific list of pages
  CSV=[]
  for i in pageList:
      n=extract_page(file,i)
      m=keywordExtractor(n)
      a,b,c=m
      #print(i)
      #print(m)
      for j in range(len(a)):
          CSV.append([i+1,a[j],b[j],c])
  return CSV

def CSVAggregator(CSV):                                    #Reduce all duplicate words into a combined output CSV of weighted frequency
  i=0
  while(i<len(CSV)-1):
    j=i+1
    while(j<len(CSV)):
        if CSV[i][1]==CSV[j][1]:
          aggFreq=(CSV[i][2]*CSV[i][3] + CSV[j][2]*CSV[j][3])/(CSV[i][3]+CSV[j][3])
          CSV[i]=[CSV[i][0],CSV[i][1],aggFreq,CSV[i][3]+CSV[j][3]]
          CSV.pop(j)
        j+=1
    i+=1

  return CSV

def CSVNormalizer(CSV,wordTotal):                         #Instead of weighing the words by total occurance on the pages they appear on, we can weight their frequencies by a specific count
    for i in CSV:
      i[2]=i[2]*i[3]/wordTotal
      i[3]=wordTotal
    return CSV

def SpecificCSV(file,word1,word2):                                  #leave either as a blank string if you dont want any extra check word
  toggleKeyword(word1)                                              #Add our specific words to the keywords we are looking for in the file, regardless of if they are frequently occuring
  toggleKeyword(word2)                                              #NEED TO DEBUG
  x=word_cooccurrence(word1,word2,file)                             #List of Pages where our requested words are present
  print("The pages that our words of interest "+ word1+" and "+ word2 +" appear on are:")
  for i in x:
    print(i+1,end=",")
  w=wordTotal(x,file)                                               #Total words on the pages where our requested words are present
  y=associatedCSVFrequency(x, file)                                 #Produce CSV of the frequent words on our requested pages
  z=CSVAggregator(y)                                                #Bunch together words that reoccur
  CSVfromSpecificPagesWithWordsOfInterest=CSVNormalizer(z,w)        #Get the frequencies for the total number of words in our concerned pages
  toggleKeyword(word1)                                              #Remove our added keyword to prevent clutter
  toggleKeyword(word2)
  return CSVfromSpecificPagesWithWordsOfInterest

def SortCSV(CSV):                     #Sorts CSVs of our format by their frequency
  sortedCSV=[]
  while(CSV):
    ifreq=[0,0,0,0]
    for i in CSV:
      if i[2]>ifreq[2]:
        ifreq=i
    sortedCSV.append(ifreq)
    CSV.remove(ifreq)
  return sortedCSV

In [ ]:
#COUNTRY SETTER

def SetCountry(country):#Write Country name, dont forget capitals first letter
  drive.mount('/content/drive')
  file = open('/content/drive/MyDrive/Sparsh_Policies/'+country+'.pdf','rb')
  return file,country;

file,country=SetCountry("Germany")
print(country)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Germany


In [ ]:
word1=""     #Try success
word2=""
x=SpecificCSV(file,word1,word2)
SortCSV(x)


In [ ]:
#NEW FUNCTIONS DEMOBLOCK
word1=""
word2=""
x=word_cooccurrence(word1,word2,file)  #Only use if you want to check the specific pages where 2 given words are coocccuring. Dont worry about capitals. If you want to check a single word/phrase, make the second word just ""
toggleKeyword(word1)                   #Including keywords forcefully in selection
toggleKeyword(word2)
print("The pages that our words of interest "+ word1+" and "+ word2 +" appear on are:")
for i in x:
  print(i+1,end=",")
print("\n---------------------")
print("The total number of words in our pages of interest are:\n")
w=wordTotal(x,file)
print(w)
# x1=word_cooccurrence("","",file)       #Sample to get total umber of words in whole doc
# w2=wordTotal(x1,file)
# print(w2)
print("---------------------")
print("Frequent words in these pages are:\n")
y=associatedCSVFrequency(x, file)
for i in y:
  print(i)
print("---------------------")
print("Aggregating and weighing duplicate words:\n")
z=CSVAggregator(y)
for i in z:
  print(i)
print("---------------------")
print("After normalizing frequency to the total number of words on these pages:\n")

f=CSVNormalizer(z,w)                    # Normalize in accordance to number of words in pages where our requested words appear
# f=CSVNormalizer(z,w2)                 # Normalize in accordance to number of words in whole doc

toggleKeyword(word1)                    #Removing added keywords for cleanup
toggleKeyword(word2)

for i in f:
  print(i)
print("---------------------")
print("Sorting all of these words by frequency:\n")
SortCSV(f)



In [ ]:
#TODO: Removal of words in title
    # (DONE) word- > list of pages where word is keyword - > relatiove frequency in all pages of words in which word is a keyword. Wieghted average or overall sum and averagess.
x=CSVFreq(file)
with open('/content/drive/MyDrive/Policy/'+country+'_Frequency_EXPERIMENTAL.csv','w+',newline='') as f:
     writer = csv.writer(f)
     writer.writerows(x)


toggleStopword(country)      #IF YOU ADD A NEW STOPWORD HERE, ADD IT AT THE BOTTOM AS WELL SO THE ARRAY SELF CORRECTS
toggleStopword('hydrogen')   #Add Hydrogen to stopwords


x=CSVFreq(file)
with open('/content/drive/MyDrive/Policy/'+country+'_Frequency(wo-Stopwords).csv','w+',newline='') as f:
     writer = csv.writer(f)
     writer.writerows(x)

toggleStopword('hydrogen')  # Remove Hydrogen from stopwords again for next file
toggleStopword(country)
